In [1]:
# Импорт библиотек
import psycopg
import pandas as pd
import mlflow
import os



/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
# Подключение к Postgres и загрузка данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20250328_430d204dda",
    "user": "mle_20250328_430d204dda",
    "password": "ebf20977880e4eb794649e9740bc4bc6",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])
connection.update(postgres_credentials)

TABLE_NAME = "users_churn"

In [ ]:
with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

# Задание 1: Сохранение названий колонок
with open("columns.txt", "w", encoding="utf-8") as f:
    f.write(",".join(df.columns))

# Задание 2: Подсчёт статистик
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}
for col in counts_columns:
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    stats.update(column_stat)

stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()
stats["unique_customers_number"] = df["customer_id"].nunique()
stats["end_date_nan"] = df["end_date"].isna().sum()

# Сохранение датафрейма в CSV
df.to_csv("users_churn.csv", index=False)

# Задание 3: Логирование в MLflow
EXPERIMENT_NAME = "churn_fio"  # Замените на ваше ФИО или уникальное название
RUN_NAME = "data_check"

# Убедитесь, что MLflow Tracking Server запущен (например, через run_mlflow_server.sh)
mlflow.set_tracking_uri("http://localhost:5000")  # Укажите URI вашего MLflow-сервера

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/experiments/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5e4dcde290>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(stats)
    mlflow.log_artifact("columns.txt", "dataframe")
    mlflow.log_artifact("users_churn.csv", "dataframe")

# Проверка завершения запуска
run = mlflow.get_run(run_id)
assert run.info.status == "FINISHED"

# Удаление временных файлов
os.remove("columns.txt")
os.remove("users_churn.csv")